In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense,Conv1D,Conv2D,Flatten,BatchNormalization,MaxPooling1D,Dropout,Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D,MaxPooling2D,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import Activation,Add,Reshape,Permute,LeakyReLU,UpSampling2D,Conv2DTranspose,Concatenate
from tensorflow.keras.layers import Lambda,InputSpec,Layer,Input,Add,ZeroPadding2D,UpSampling2D,MaxPooling2D,Conv2D,Bidirectional,LSTM
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau,TensorBoard
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.utils import resample
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.optimizers import Adam,SGD
import datetime
from tensorflow.keras.utils import plot_model
import os
import flask
import json


# def load_dataset():

ddata = pd.read_pickle('./data/test_labels.pkl')
ddata['Label'].value_counts()

In [ ]:
# ddata.info()
ddata.drop(['Timestamp'],axis=1,inplace=True)

# DATA

In [ ]:
df = ddata
# df.drop(['fwd_header_length'], axis=1, inplace=True)

In [ ]:
# drop columns
# df.drop(['protocol'], axis=1, inplace=True)
# df.drop(['Dst Port'], axis=1, inplace=True)

In [ ]:
df.describe(include=[int, float])
df.describe(include=[object]).transpose()

# Dealing with duplicates

In [ ]:
df.duplicated().any()

In [ ]:
print('Data size BEFORE deleteting instances with duplicate values: ', df.shape[0], end='\n\n')

# Remove duplicate rows
df.drop_duplicates(inplace=True, keep=False, ignore_index=True)

print('Data size AFTER deleteting instances containing duplicate values: ', df.shape[0])

Dealing with missing values

In [ ]:
df.isnull().sum().sum()
df.isnull().sum() / df.shape[0]

In [ ]:
df.columns[df.isnull().any()]

In [ ]:
print('Data size BEFORE deleteting instances with missing values: ', df.shape[0], end='\n\n')

# Remove missing values
df.dropna(axis=0, inplace=True, how="any")

print('Data size AFTER deleteting instances containing missing values: ', df.shape[0])

# Dealing with infinite values

In [ ]:
# Checking if all values are finite.
np.all(np.isfinite(df.drop(['Label'], axis=1)))

In [ ]:
# Replace infinite values to NaN
df.replace([-np.inf, np.inf], np.nan, inplace=True)

# Check which labels are related to infinte values
df[(df['Flow Byts/s'].isnull()) & (df['Flow Pkts/s'].isnull())].Label.unique()


In [ ]:
print('Data size BEFORE deleteting instances with infinite values: ', df.shape[0], end='\n\n')

# Remove infinte values
df.dropna(axis=0, how='any', inplace=True)

print('Data size AFTER deleteting instances containing infinite values: ', df.shape[0])

# Dealing with features with quasi null std deviation

In [ ]:
dataset_std = df.std(numeric_only=True)
dataset_std

In [ ]:
# Find Features that meet the threshold
constant_features = [column for column, std in dataset_std.iteritems() if std < 0.01]

# Drop the constant features
df.drop(labels=constant_features, axis=1, inplace=True)
print(constant_features)

# Observing the outliers

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# Identifying outliers with interquartile range
filt = (df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))
print(filt.sum())

In [ ]:
fig = plt.figure(figsize=(15, 8))
sns.boxplot(data=df[['Pkt Size Avg', 'Bwd Seg Size Avg']], orient="h")

#plt.title('Summary of some variables containing outliers', fontsize=18)
plt.show()
fig.savefig(os.path.join('./images', 'outliers.pdf'))

Convert the dtype of some features

In [ ]:
df[['Flow Byts/s', 'Flow Pkts/s']] = df[['Flow Byts/s', 'Flow Pkts/s']].apply(pd.to_numeric)

Create a new feature `Port Category`

In [ ]:
# conds = [
#     (df['dst_port'] >= 1) & (df['dst_port'] < 1024),
#     (df['dst_port'] >= 1024) & (df['dst_port'] < 49152),
#     (df['dst_port'] >= 49152) & (df['dst_port'] <= 65535)
# ]

# choices = [
#     "1 - 1023", 
#     "1024 - 49151",
#     "49152 - 65535"
# ]

# df.insert(1, 'destination_port_category', np.select(conds, choices, default="0"))

# Data Exploration

### Correlation Matrix

In [ ]:
dataset_corr = df.corr()
dataset_corr.head(5)

In [ ]:
fig = plt.figure(figsize=(15, 15))
sns.set(font_scale=1.0)
ax = sns.heatmap(dataset_corr, annot=False)
fig.savefig(os.path.join('./images', 'correlation matrix.pdf'))

In [ ]:
# Create & Apply mask
mask = np.triu(np.ones_like(dataset_corr, dtype=bool))
tri_df = dataset_corr.mask(mask)

# Find Features that meet the threshold
correlated_features = [c for c in tri_df.columns if any(tri_df[c] > 0.98)]
print(correlated_features)
# Drop the highly correlated features
df.drop(labels=correlated_features, axis=1, inplace=True)

# Information about the data

In [ ]:
# drop columns
print(df.info())
train_dataset = df
print(train_dataset['Label'].value_counts())
# df.to_csv('./final_1/final_had_p.csv',index=False)

# Port Usage Comparaison

In [ ]:
# fig, ax = plt.subplots(figsize=(10,10))

# benign_ports = df.loc[df['Label'] == 'Benign', 'destination_port_category']
# malicious_ports = df.loc[df['Label'] != 'Benign', 'destination_port_category']

# # get rid of rows with specific value
# benign_ports = benign_ports[benign_ports != "0"]
# malicious_ports = malicious_ports[malicious_ports != "0"]

# # sum each port category column
# benign_ports = benign_ports.value_counts()
# malicious_ports = malicious_ports.value_counts()

# indexes = np.arange(3)
# width = 0.4
# rect1 = plt.bar(indexes, benign_ports.values, width, color="steelblue", label="benign")
# rect2 = plt.bar(indexes + width, malicious_ports.values, width, color="indianred", label="malicious")

# def add_text(rect):
#     # add text to top of each bar
#     for r in rect:
#         h = r.get_height()
#         plt.text(r.get_x() + r.get_width()/2, h*1.01, s=format(h, ",") ,fontsize=12, ha='center', va='bottom')

# add_text(rect1)
# add_text(rect2)

# ax.set_xticks(indexes + width / 2)
# ax.set_xticklabels(["1 - 1,023", "1,024 - 49,151", "49,152 - 65,535"])
# plt.title('Distribution of Port Usage\nAccording to Network Activity Type')
# plt.xlabel('Port Range')
# plt.ylabel('Frequency')
# plt.legend()
# plt.grid(True)
# plt.show()
# fig.savefig(os.path.join('./images/', 'port_usage_comparison.pdf'))

# Label encoding

In [ ]:
train_dataset['Label'].value_counts()
# encode label
labelE = LabelEncoder()
train_dataset['Label'] = labelE.fit_transform(train_dataset['Label'])
train_dataset['Label'].value_counts()
y = train_dataset['Label']
train_dataset = train_dataset.drop(['Label'],axis=1)
# train_dataset = train_dataset.drop(['destination_port_category'],axis=1)


In [ ]:
# df.drop(['destination_port_category'], axis=1, inplace=True)

# Normalization

In [ ]:
# normalize data
# sc = StandardScaler()
# train_dataset = sc.fit_transform(train_dataset)
# train_dataset = pd.DataFrame(train_dataset,columns=df.columns[:-1])

# Train Test Split

In [ ]:
# split data
X_train, X_test, y_train, y_test = train_test_split(train_dataset, y, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train.value_counts())

# Scaling features to a range

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, QuantileTransformer
from sklearn.compose import ColumnTransformer

In [ ]:
categorical_features = train_dataset.select_dtypes(exclude=["int64", "float64"]).columns
numeric_features = train_dataset.select_dtypes(exclude=[object]).columns

preprocessor = ColumnTransformer(transformers=[
    ('categoricals', OneHotEncoder(drop='first', sparse=False, handle_unknown='error'), categorical_features),
    ('numericals', QuantileTransformer(), numeric_features)
])

In [ ]:
print(categorical_features)
print(numeric_features)

Preprocess the features

In [ ]:
columns = numeric_features.tolist()
X_train 
from pickle import dump
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=columns)
dump(preprocessor, open('./normalization/preprocessor.pkl', 'wb'))
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=columns)
X_val = pd.DataFrame(preprocessor.transform(X_val), columns=columns)

In [ ]:
DATA_DIR = './final_1M/'
X_train.to_pickle(os.path.join(DATA_DIR,  'train/train_features.pkl'))
X_val.to_pickle(os.path.join(DATA_DIR,  'val/val_features.pkl'))
X_test.to_pickle(os.path.join(DATA_DIR,  'test/test_features.pkl'))

y_train.to_pickle(os.path.join(DATA_DIR,  'train/train_labels.pkl'))
y_val.to_pickle(os.path.join(DATA_DIR,  'val/val_labels.pkl'))
y_test.to_pickle(os.path.join(DATA_DIR,  'test/test_labels.pkl'))

# Balance the training set using combination of SMOTE & RandomUnderSampler

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

def balance_dataset(X, y, undersampling_strategy, oversampling_strategy):

    under_sampler = RandomUnderSampler(sampling_strategy=undersampling_strategy, random_state=0)
    X_under, y_under = under_sampler.fit_resample(X, y)

    over_sampler = SMOTE(sampling_strategy=oversampling_strategy)
    X_bal, y_bal = over_sampler.fit_resample(X_under, y_under)
    
    return X_bal, y_bal

In [ ]:
undersampling_strategy = {
    0    :704017,
    3    :600136,
    1    :600098,
    2    :593761
}

oversampling_strategy = {
    0    :704017,
    3    :600136,
    1    :600098,
    2    :593761
}

# Balance the training set
X_train_bal, y_train_bal = balance_dataset(X_train, y_train, undersampling_strategy, oversampling_strategy)

# Save the balanced training set
X_train_bal.to_pickle(os.path.join(DATA_DIR,  'train/train_features_balanced.pkl'))
y_train_bal.to_pickle(os.path.join(DATA_DIR,  'train/train_labels_balanced.pkl'))

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

# sum each port category column
imbalanced = y_train.value_counts()
balanced = y_train_bal.value_counts()

indexes = np.arange(4)
width = 0.4
rect1 = plt.bar(indexes, imbalanced.values, width, color="steelblue", label="imbalanced")
rect2 = plt.bar(indexes + width, balanced.values, width, color="indianred", label="balanced")

def add_text(rect):
    """Add text to top of each bar."""
    for r in rect:
        h = r.get_height()
        plt.text(r.get_x() + r.get_width()/2, h*1.01, s=format(h, ",") ,fontsize=12, ha='center', va='bottom')

add_text(rect1)
add_text(rect2)

ax.set_xticks(indexes + width / 2)
ax.set_xticklabels(['PortScan', 'DDoS',  'passwork_attack','Benign'])
plt.xlabel('Traffic Activity', fontsize=16)
plt.ylabel('# instances', fontsize=16)
plt.legend()
plt.grid()
plt.show()
fig.savefig(os.path.join('./images/', 'balanced_dataset.pdf'))

In [ ]:
def load_pkl():
    X_train = pd.read_pickle(os.path.join(DATA_DIR, 'train/train_features_balanced.pkl'))
    y_train = pd.read_pickle(os.path.join(DATA_DIR, 'train/train_labels_balanced.pkl'))
    X_val = pd.read_pickle(os.path.join(DATA_DIR, 'val/val_features.pkl'))
    y_val = pd.read_pickle(os.path.join(DATA_DIR, 'val/val_labels.pkl'))
    X_test = pd.read_pickle(os.path.join(DATA_DIR, 'test/test_features.pkl'))
    y_test = pd.read_pickle(os.path.join(DATA_DIR, 'test/test_labels.pkl'))
    return X_train,y_train,X_test,y_test,X_val,y_val

def re_shape_2D(X_train,X_test,y_train,y_test,X_val,y_val):
    X_train = X_train.to_numpy().reshape(len(X_train),X_train.shape[1]//8,8,1)
    X_test = X_test.to_numpy().reshape(len(X_test),X_test.shape[1]//8,8,1)
    X_val = X_val.to_numpy().reshape(len(X_val),X_val.shape[1]//8,8,1)
    return X_train,X_test,y_train,y_test,X_val,y_val

def re_shape_1D(X_train,X_test,y_train,y_test,X_val,y_val):
    X_train = X_train.to_numpy().reshape(len(X_train),X_train.shape[1],1)
    X_test = X_test.to_numpy().reshape(len(X_test),X_test.shape[1],1)
    X_val = X_val.to_numpy().reshape(len(X_val),X_val.shape[1],1)
    return X_train,X_test,y_train,y_test,X_val,y_val

import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

# define cnn model
def define_model_99(time,Xtrain,ytrain,Xtest,ytest,logdir,epochs=10,batch_size=128,lr =0.001):
    # load model
    model = Sequential()
    model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=(Xtrain.shape[1],Xtrain.shape[2],Xtrain.shape[3]),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='glorot_uniform'))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(4,activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    # create data generator 
    time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    logdir = logdir + time
    tensorboard_callback = TensorBoard(log_dir=logdir)
    calls = [tensorboard_callback,
            #  EarlyStopping(monitor='val_loss',patience=5,verbose=1,mode='auto'),
            #  ModelCheckpoint(filepath=f'models/{time}/model_{time}.h5',monitor='val_loss',save_best_only=True,mode='auto'),
            #  ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=3,verbose=1,mode='auto',min_delta=0.0001,cooldown=0,min_lr=0)
            ]
    model.fit(Xtrain,ytrain,epochs=epochs,batch_size=batch_size,validation_data=(Xtest,ytest),callbacks=calls,verbose=1)
    return model

def multi_head(time,Xtrain,ytrain,Xtest,ytest,Xval,yval,logdir,epochs=10,batch_size=128,lr =0.001):
    input1s = Input(shape=(Xtrain.shape[1],1))
    conv1d_1s1 = Conv1D(filters=32,kernel_size=3,activation='relu',padding='same')(input1s)
    batch_1s1 = BatchNormalization()(conv1d_1s1)
    maxpool_1s1 = MaxPooling1D(pool_size=3,strides=2,padding='same')(batch_1s1)
    conv1d_1s2 = Conv1D(filters=32,kernel_size=3,activation='relu',padding='same')(maxpool_1s1)
    batch_1s2 = BatchNormalization()(conv1d_1s2)
    maxpool_1s2 = MaxPooling1D(pool_size=3,strides=2,padding='same')(batch_1s2)
    flat1 = Flatten()(maxpool_1s2)

    input2s = Input(shape=(Xtrain.shape[1],1))
    conv1d_2s1 = Conv1D(filters=64,kernel_size=5,activation='relu',padding='same')(input2s)
    batch_2s1 = BatchNormalization()(conv1d_2s1)
    maxpool_2s1 = MaxPooling1D(pool_size=3,strides=2,padding='same')(batch_2s1)
    conv1d_2s2 = Conv1D(filters=64,kernel_size=5,activation='relu',padding='same')(maxpool_2s1)
    batch_2s2 = BatchNormalization()(conv1d_2s2)
    maxpool_2s2 = MaxPooling1D(pool_size=3,strides=2,padding='same')(batch_2s2)
    flat2 = Flatten()(maxpool_2s2)
    
    input3s = Input(shape=(Xtrain.shape[1],1))
    conv1d_3s1 = Conv1D(filters=128,kernel_size=7,activation='relu',padding='same')(input3s)
    batch_3s1 = BatchNormalization()(conv1d_3s1)
    maxpool3 = MaxPooling1D(pool_size=3,strides=2,padding='same')(batch_3s1)
    conv1d_3s2 = Conv1D(filters=128,kernel_size=7,activation='relu',padding='same')(maxpool3)
    batch_3s2 = BatchNormalization()(conv1d_3s2)
    maxpool_3s2 = MaxPooling1D(pool_size=3,strides=2,padding='same')(batch_3s2)
    flat3 = Flatten()(maxpool_3s2)

    concat = concatenate([flat1,flat2,flat3])
    dense1 = Dense(256,activation='relu')(concat)
    drop = Dropout(0.2)(dense1)
    dense2 = Dense(4,activation='softmax')(drop)
    model = Model(inputs=[input1s,input2s,input3s],outputs=dense2)
    #plot model
    plot_model(model, to_file=f'models/{time}/model_{time}.png', show_shapes=True, show_layer_names=True)

    opt = SGD(lr=lr, momentum=0.9)  
    model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

    time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    logdir = logdir + time
    tensorboard_callback = TensorBoard(log_dir=logdir)
    calls = [tensorboard_callback,
             EarlyStopping(monitor='val_loss',patience=5,verbose=1,mode='auto'),
             ModelCheckpoint(filepath=f'models/{time}/model_{time}.h5',monitor='val_loss',save_best_only=True,mode='auto'),
             ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=3,verbose=1,mode='auto',min_delta=0.0001,cooldown=0,min_lr=0)
            ]
    model.fit([Xtrain,Xtrain,Xtrain],ytrain,epochs=epochs,batch_size=batch_size,validation_data=([Xtest,Xtest,Xtest],ytest),callbacks=[tensorboard_callback],verbose=1)
    # _,acc = model.evaluate([Xtest,Xtest,Xtest],ytest,verbose=0)
    # print("Accuracy: %.2f%%" % (acc*100))
    return model
def multi_head1(time,Xtrain,ytrain,Xtest,ytest,Xval,yval,logdir,epochs=10,batch_size=128,lr =0.001):
    input1s = Input(shape=(Xtrain.shape[1],1))
    conv1d_1s1 = Conv1D(filters=32,kernel_size=3,activation='relu',padding='same')(input1s)
    batch_1s1 = BatchNormalization()(conv1d_1s1)
    maxpool_1s1 = MaxPooling1D(pool_size=3,strides=2,padding='same')(batch_1s1)
    flat1 = Flatten()(maxpool_1s1)

    input2s = Input(shape=(Xtrain.shape[1],1))
    conv1d_2s1 = Conv1D(filters=64,kernel_size=5,activation='relu',padding='same')(input2s)
    batch_2s1 = BatchNormalization()(conv1d_2s1)
    maxpool_2s1 = MaxPooling1D(pool_size=3,strides=2,padding='same')(batch_2s1)
    flat2 = Flatten()(maxpool_2s1)
    
    input3s = Input(shape=(Xtrain.shape[1],1))
    conv1d_3s1 = Conv1D(filters=128,kernel_size=7,activation='relu',padding='same')(input3s)
    batch_3s1 = BatchNormalization()(conv1d_3s1)
    maxpool3 = MaxPooling1D(pool_size=3,strides=2,padding='same')(batch_3s1)
    flat3 = Flatten()(maxpool3)

    concat = concatenate([flat1,flat2,flat3])
    dense1 = Dense(256,activation='relu')(concat)
    drop = Dropout(0.2)(dense1)
    dense2 = Dense(4,activation='softmax')(drop)
    model = Model(inputs=[input1s,input2s,input3s],outputs=dense2)
    #plot model
    plot_model(model, to_file=f'models/{time}/model_{time}.png', show_shapes=True, show_layer_names=True)
    
    opt = SGD(lr=lr, momentum=0.9)  
    model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

    time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    logdir = logdir + time
    tensorboard_callback = TensorBoard(log_dir=logdir)
    calls = [tensorboard_callback,
             EarlyStopping(monitor='val_loss',patience=5,verbose=1,mode='auto'),
             ModelCheckpoint(filepath=f'models/{time}/model_{time}.h5',monitor='val_loss',save_best_only=True,mode='auto'),
             ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=3,verbose=1,mode='auto',min_delta=0.0001,cooldown=0,min_lr=0)
            ]
    model.fit([Xtrain,Xtrain,Xtrain],ytrain,epochs=epochs,batch_size=batch_size,validation_data=([Xval,Xval,Xval],yval),callbacks=[tensorboard_callback],verbose=1)
    _,acc = model.evaluate([Xtest,Xtest,Xtest],ytest,verbose=0)
    print("Accuracy: %.2f%%" % (acc*100))
    return model

def CNN_normal(time,Xtrain,ytrain,Xtest,ytest,logdir,epochs=10,batch_size=128,lr =0.001):
    # define model conv2D
    input1s = Input(shape=(Xtrain.shape[1],Xtrain.shape[2],Xtrain.shape[3]))
    conv1 = Conv2D(32,(3,3),activation='relu',padding='same')(input1s)
    conv2 = Conv2D(32,(3,3),activation='relu',padding='same')(conv1)
    bat = BatchNormalization()(conv2)
    pool1 = MaxPooling2D((3,3))(bat)
    flat1 = Flatten()(pool1)
    dense1 = Dense(256,activation='relu')(flat1)
    drop = Dropout(0.2)(dense1)
    dense2 = Dense(4,activation='softmax')(drop)
    model = Model(inputs=input1s,outputs=dense2)
    #plot model
    plot_model(model, to_file=f'models/{time}/model_{time}.png', show_shapes=True, show_layer_names=True)

    opt = SGD(lr=lr, momentum=0.9)  
    model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

    time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    logdir = logdir + time
    tensorboard_callback = TensorBoard(log_dir=logdir)
    calls = [tensorboard_callback,
            EarlyStopping(monitor='val_acc', patience=2, verbose=1, mode='auto'),
            ModelCheckpoint(filepath=f'./models/{time}/model_checkpoint{time}.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
            #  ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=3,verbose=1,mode='auto',min_delta=0.0001,cooldown=0,min_lr=0)
            ]
    model.fit(Xtrain,ytrain,epochs=epochs,batch_size=batch_size,validation_data=(Xtest,ytest),callbacks=calls,verbose=1)
    return model

In [ ]:
X_train,y_train,X_test,y_test,X_val,y_val = load_pkl()
X_train,X_test,y_train,y_test,X_val,y_val = re_shape_1D(X_train,X_test,y_train,y_test,X_val,y_val)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# %tensorboard --logdir logs/fit
time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
os.mkdir(f'./models/{time}')
logdir="logs/multi/" + time +'/'
model = multi_head1(time,X_train,y_train,X_test,y_test,X_val,y_val,logdir,10,128)
model.save(f'models/{time}/model_{time}.h5')

In [ ]:
da = pd.DataFrame(y_test)
da.value_counts()

In [ ]:
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])